In [2]:
import tensorflow as tf
import numpy as np
import shelve
import joblib
from math import ceil
import time
import layers
import model
from Augment import valid_datagen
import helper
from test_config import vocabulary,test_batch_size,mount_point
from Arch import CNN

In [11]:
with shelve.open(mount_point+'IAM_Data','c') as shelf:
    valid_label = shelf['valid_label']

valid_array = joblib.load(mount_point+'data/valid_array')

In [12]:
graph,dropout_lstm,dropout_fc,inputs,time_steps,targets,loss,train,decoded,label_error_rate,seq_len,is_training,conv_dropout,gradients,interim_dropout= model.model()

valid_generator = valid_datagen.flow(valid_array,valid_label,valid_batch_size)

num_valid_samples = valid_array.shape[0]

num_vbatches = 1

++Conv layer info info:
Num_filters =  64
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 2)
++Conv layer info info:
Num_filters =  128
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_1:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 2)
++Conv layer info info:
Num_filters =  256
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_2:0", shape=(), dtype=float32)
++Conv layer info info:
Num_filters =  256
Kernel_size =  3
Strides =  1
activation =  leaky_relu
is_training =  Tensor("Placeholder:0", dtype=bool)
dropout =  Tensor("strided_slice_3:0", shape=(), dtype=float32)
--Max_pool info:
Strides =  (3, 1)
++Conv layer info info:
Num_filters =  512
Ker

INFO:tensorflow:Summary name stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/kernel:0-weight is illegal; using stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/kernel_0-weight instead.
INFO:tensorflow:Summary name stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/bias:0-weight is illegal; using stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/lstm_cell/bias_0-weight instead.
INFO:tensorflow:Summary name dense/kernel:0-weight is illegal; using dense/kernel_0-weight instead.
INFO:tensorflow:Summary name dense/bias:0-weight is illegal; using dense/bias_0-weight instead.


In [13]:
infer_inputs = []; infer_outputs = []; infer_targets = []

In [15]:
with tf.Session(graph = graph) as sess:
        
    saver = tf.train.Saver(max_to_keep=None)
    
    sess.run(tf.global_variables_initializer())
    timer  = 0

    saver.restore(sess, mount_point+'saved_models_ler0.07_leakyrelu/cnn_lstm_fc_'+str(resume_epoch))

    start_time = time.time()

    count = 0

    for x,y in valid_generator:
       
        infer_inputs.append(x)
        infer_targets.append(y)
        
        actual_batch_size = x.shape[0]

        if count == num_vbatches:
            break

        y,widths = np.hsplit(y,2)

        widths = np.squeeze(widths,axis=1)
        y = np.squeeze(y,axis=1)

        widths = [layers.calc_out_dims(CNN,0,int(width))[1] for width in widths]
        widths = np.array(widths)
 
        sparse_y = helper._batch_y(y,vocabulary)
            
        feed = {
                         inputs:x,targets:sparse_y,
                         time_steps:widths,conv_dropout:[1]*len(CNN),
                         dropout_fc:1,dropout_lstm:1,interim_dropout:1,
                         is_training:False
                }
        
        loss_val,d = sess.run([loss,decoded],feed_dict=feed)
        infer_outputs.append(d)
        count+=1

INFO:tensorflow:Restoring parameters from ../saved_models_ler0.07_leakyrelu/cnn_lstm_fc_37


In [16]:
print(loss_val)

11.122264


In [18]:
content = "".join([vocabulary[char] for char in infer_outputs[0][0][1]])

In [19]:
print(content)

the Navy - why should I pick on Brikes ? " 


In [20]:
img1 = infer_inputs[0][0]
img1 = img1.reshape(img1.shape[:2])

In [21]:
img1.shape

(114, 758)

In [22]:
plt.imsave('test_input.jpg',img1,cmap='gray',format='jpg')

In [23]:
infer_targets[0][0]

array(['the Navy - why should I pick on Pericles ? " ', '570'],
      dtype='<U73')